In [ ]:
# Experimentación: Búsqueda con Multithreading
import os
import time
from threading import Thread

# Configuración del experimento
DATASET_PATH = "dataset"
SEARCH_TERM = "needle"
REPEATS = 10  # Número de repeticiones por prueba


def search_in_file(file_path, search_term):
    """Realiza una búsqueda del término en un archivo dado."""
    with open(file_path, "r") as f:
        for line in f:
            if search_term in line:
                return True
    return False


def threaded_search(file_paths, search_term):
    """Realiza una búsqueda paralela con multithreading."""
    def thread_task(file_path, results, index):
        results[index] = search_in_file(file_path, search_term)

    results = [None] * len(file_paths)
    threads = []
    for i, file_path in enumerate(file_paths):
        thread = Thread(target=thread_task, args=(file_path, results, i))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()
    return results


def measure_execution_time(file_paths, search_term, repeats=1):
    """Mide el tiempo de ejecución de la búsqueda con multithreading."""
    total_time = 0
    for _ in range(repeats):
        start_time = time.time()
        threaded_search(file_paths, search_term)
        total_time += time.time() - start_time
    return total_time / repeats


def run_experiment():
    """Ejecuta el experimento con multithreading y registra los resultados."""
    results = []

    for folder in os.listdir(DATASET_PATH):
        folder_path = os.path.join(DATASET_PATH, folder)
        if not os.path.isdir(folder_path):
            continue
        
        file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path)]
        size, num_files = map(int, [s.split("_")[1].replace("MB", "") for s in folder.split("_")[1::2]])
        
        thread_time = measure_execution_time(file_paths, SEARCH_TERM, REPEATS)

        results.append({
            "file_size_mb": size,
            "num_files": num_files,
            "threaded_time": thread_time
        })

    return results


def main():
    """Punto de entrada principal para el experimento de multithreading."""
    print("Ejecutando experimentos con Multithreading...")
    results = run_experiment()
    print("Resultados:")
    for result in results:
        print(result)


if __name__ == "__main__":
    main()
